# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find out whether moving to particular location in toronto is good or not while getting job offer on particular neighbour hood
We will use our data science powers to generate a few most promissing neighborhoods based on this criteria.
Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.



## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood 
* number of and distance to GYM or school or park 
* distance of neighborhood from city center
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood=pd.DataFrame(df[0])

In [2]:
neighborhood.head()

,0,1,2
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [3]:
neighborhood.shape

(181, 3)

In [4]:
neighborhood.columns =['Postal code', 'Borough','Neighborhood']

In [5]:
neighborhood.head()

,Postal code,Borough,Neighborhood
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [6]:
toronto_df=neighborhood.drop([neighborhood.index[0]])

In [7]:
toronto_df.head()

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
toronto_df.shape

(180, 3)

In [9]:
toronto_df.drop(toronto_df.index[toronto_df['Borough'] =="Not assigned"], inplace = True)

In [10]:
toronto_df.shape

(103, 3)

In [11]:
geodf=pd.read_csv("Geospatial_Coordinates.csv")

In [12]:
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
geodf.rename({'Postal Code': 'Postal code', 'Latitude': 'Latitude','Longitude':'Longitude'}, axis=1, inplace=True)

In [14]:
geodf.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
result = pd.merge(toronto_df, geodf)

In [16]:
result.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [17]:
result1=result.iloc[:,1:]

In [18]:
result1.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [19]:
result1.shape

(103, 4)

In [20]:
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [21]:
address = 'Downtown Toronto, DT'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 44.3856315, -79.6913776.


## Methodology <a name="methodology"></a>

we will be using foursqaure API to find out near by park , school or GYm or coffee shop or any good market place so that based on this finding
new joining will be taking decision whether that person will be shifting to that particular location or not as 
when he/she get the joining offer from any company


## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis

In [22]:
# create map of Downtown Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result1['Latitude'], result1['Longitude'], result1['Borough'], result1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [23]:
CLIENT_ID = 'JAF1W505F51KJWCELTECRVWBLH1WGV5YUS2K0QUSM2RQI15B'
CLIENT_SECRET = 'NA5JBMTQH3CZVRU0D5YNPBUWBWFPXLWO1DDZ0V34SDUHM0LO'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JAF1W505F51KJWCELTECRVWBLH1WGV5YUS2K0QUSM2RQI15B
CLIENT_SECRET:NA5JBMTQH3CZVRU0D5YNPBUWBWFPXLWO1DDZ0V34SDUHM0LO


In [24]:
Downtown_Toronto_data = result1[result1['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [25]:
Downtown_Toronto_data.loc[0, 'Neighborhood']

neighborhood_latitude = Downtown_Toronto_data.loc[0, 'Latitude'] 
neighborhood_longitude = Downtown_Toronto_data.loc[0, 'Longitude']

neighborhood_name = Downtown_Toronto_data.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park / Harbourfront are 43.6542599, -79.3606359.


In [27]:
Downtown_Toronto_data.loc[0, 'Neighborhood']

'Regent Park / Harbourfront'

In [28]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JAF1W505F51KJWCELTECRVWBLH1WGV5YUS2K0QUSM2RQI15B&client_secret=NA5JBMTQH3CZVRU0D5YNPBUWBWFPXLWO1DDZ0V34SDUHM0LO&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [29]:
import json

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e89b64b923935001bd69180'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
from pandas.io.json import json_normalize

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


## Results and Discussion <a name="results"></a>

Based on nearby store  park , Restaurant and  coffee shop , new joiner will decide whethere he/she has to move that place or not

## Conclusion <a name="conclusion"></a>

purpose of this project to help the new joiner who is moving into Toronto city and it help him/her to identify best place near by and 
base on Data new joiner whill take a decision to shift that particular location or not